In [1]:
#! /usr/bin/env python
import numpy as np
import argparse
import os
import sys




input_file = "LOCPOT"
fermi_e = 0
output_file = 'output.dat'
visualization = False
igor = False
igor_output = "Planar.itx"
direction = 'z'
vac_con_Ediff = 1E-3
vac_con_width = 3


def mkdir(dirname):
    if not os.path.exists(os.path.dirname(dirname+"/")):
        os.makedirs(os.path.dirname(dirname+"/"))


def read_CHGCAR(ipf="LOCPOT",direction="Z"):
    ip=open(ipf,'r')
    print ('* Name of System : ' + ip.readline())
    ip.readline()
    cell_vec=[]
    cell_vec.append(ip.readline().split())
    cell_vec.append(ip.readline().split())
    cell_vec.append(ip.readline().split())
    cell_vec=np.array(cell_vec,dtype="d")
    species=ip.readline().split()
    natoms=list(map(lambda x:int(x),ip.readline().split()))
    tot_natoms=sum(natoms)
    coord_read=ip.readline()
    if coord_read.startswith("D") or coord_read.startswith("d"):
        coord_type="Direct"
    elif coord_read.startswith("F") or coord_read.startswith("f"):
        coord_type="Direct"
    elif coord_read.startswith("C") or coord_read.startswith("c"):
        coord_type="Cartesian"
    else:
        print("------- Coordination Type Recognition Failed. Calculation ended. -------")
        sys.exit(1)

    coord=[]
    for i in range(tot_natoms):
        coord.append(list(map(lambda x:float(x),ip.readline().split())))
    ip.readline()
    grids = list(map(lambda x:int(x),ip.readline().split()))
    nx, ny, nz = grids
    print("* Matrix : [ %d ] x [ %d ] x [ %d ]"%(nx, ny, nz))
    lines = ip.readlines()

#------------------------------------------------------------
# From CHG.py (WS)
    pot = []
    for x in lines:
        if "a" in x:
            break
        else:
            tmp = x.split()
            for y in tmp:
                pot.append(y)
    pot = np.reshape(np.array(pot, dtype="d"), (grids[::-1]))
    # if you want to undo reshape, use command, pot.flatten()
#------------------------------------------------------------
    if direction in "Z":
    # since chglist is nz ny nx sequence, so if we want nz direction, mean axis will be axis=(1,2)
    # if axis is in tuple type, they calculate average in both direction.
        potavg=np.mean(pot, axis=(1,2))
    if direction in "Y":
        potavg=np.mean(pot, axis=(2,0))
    if direction in "X":
        potavg=np.mean(pot, axis=(0,1))
    ip.close()
    return [cell_vec, species, natoms, coord_type, coord], grids, pot, potavg



def write_Igor2d(x,y,lLabel, bLabel, output, axis_name=""):
    '''x and y is raw data (np.array type).
    output is name of output file.'''
    if os.path.exists(output):
        print("%s already exists. File substituted to new file."%(igor_output))
    itx=open(output,'w')
    itx.write("IGOR\n")
    itx.write("WAVES/D x_%s %s\n"%(axis_name,axis_name))
    itx.write("BEGIN\n")
    for (x1,y1) in zip(x,y):
        itx.write(str(x1)+" "+str(y1)+"\n")
    itx.write("END\n")
    itx.write("X Display %s vs x_%s as \"planar_average_%s\" \n"%(axis_name,axis_name,axis_name))
    itx.write("X DefaultFont/U \"Times New Roman\"\n")
    itx.write("X ModifyGraph marker=19\n")
    itx.write("X ModifyGraph tick=2\n")
    itx.write("X ModifyGraph mirror=1\n")
    itx.write("X ModifyGraph fSize=28\n")
    itx.write("X ModifyGraph lblMargin(left)=15,lblMargin(bottom)=10\n")
    itx.write("X ModifyGraph standoff=0\n")
    itx.write("X ModifyGraph axThick=1.5\n")
    itx.write("X ModifyGraph axisOnTop=1\n")
    itx.write("X ModifyGraph width=453.543,height=340.157\n")
    itx.write("X ModifyGraph zero(left)=8\n")
    itx.write("X ModifyGraph zero(bottom)=1\n")
    itx.write("X "+lLabel)
    itx.write("X ModifyGraph axThick=2\n")
    itx.write("X ModifyGraph lsize=2\n")
    itx.write("X ModifyGraph lblMargin(left)=5\n")
    itx.write("X "+bLabel)
    itx.close()

def extract(keyword, file='OUTCAR'):
    '''Get Fermi Energy Energy from OUTCAR'''
    with open(file, 'r') as out:
        for line in out:
            if keyword in line:
                return line.split()

def length(a):
    return (a[0]**2+a[1]**2+a[2]**2)**(0.5)

def dir2car(coord,cell_vec):
    '''Direct to Cartesian (function from gycoco.py)
    [input] : coord, cell_vec
    |-> coord : coordination. @np.array(dtype="d")
    |-> cell_vec : cell vector. @np.array(dtype='d')
    [output] : new coordination @np.array(dtype='d')
    '''
    return np.matmul(coord,cell_vec)


In [3]:



#------------------------------------------------------------------
# Starts Script.
#-----------------------------------------------------------------
#------------------------------------------------------------------
# input Direction testing
#------------------------------------------------------------------

if direction in "zZcC3":
    direction="Z"
    ndir=3
elif direction in "yYbB2":
    direction="Y"
    ndir=2
elif direction in "xXaA1":
    direction="X"
    ndir=1
else:
    raise IOError("------- Wrong input of direction. Please set direction again -------")
    sys.exit(1)

#------------------------------------------------------------------
# All files are generated in this directory.
#------------------------------------------------------------------
directory="wf_cal"
mkdir(directory)
log=open(directory+'/workfunction.log','w')

#------------------------------------------------------------------
# Extract fermi energy if 'OUTCAR' file exists in same directory
#------------------------------------------------------------------
if os.path.exists(input_file):
    print("--------------        {} exists. Calculation starts.        --------------".format(input_file))
    log.write(input_file+" exists. Calculation starts\n")
    log.write("--------------------\n")
    log.write("input file = "+input_file+'\n')
    log.write("output file = "+output_file+'\n')

    #------------------------------------------------------------------
    # Get the potential
    #-----------------------------------------------------------------
    values=read_CHGCAR(input_file, direction)
    potavg, grids, cell_vec = values[3], np.array(values[1],dtype='d'), np.array(values[0][0],dtype='d')
    cell_vec_length = np.array((length(values[0][0][0]), length(values[0][0][1]), length(values[0][0][2])),dtype='d')
    resolution=cell_vec_length/grids

    #------------------------------------------------------------------
    # Get E_vacuum from average potential along the direction.
    #------------------------------------------------------------------
    potavg_temp=np.append(np.array([0],dtype='d'),potavg[:-1])
    diff_potavg=abs(potavg-potavg_temp)


    log.write("--------------------\n")
    print(">>> Vacuum Level Searching",file=log)
    print("|---> E_diff for vacuum convergence is %f"%(vac_con_Ediff),file=log)


    temp, x_temp =[], []
    for i in range(len(diff_potavg)):
        if diff_potavg[i]<(vac_con_Ediff):
            temp.append(potavg[i])
            x_temp.append(i)
    x_temp=np.array(x_temp,dtype='d')
    coord=np.array(values[0][4],dtype='d')
    

    if len(x_temp)==0:
        print("|---> No Flat region was found. Maybe Dipole correction or Increasing vacuum level is needed.", file=log)
    else:
        diff_x_temp=x_temp-np.append(np.array([0],dtype='d'),x_temp[:-1])
        n=0
        for i in diff_x_temp[1:]:
            if i!=1:
                n+=1
    if n==0:
        E_vac=np.mean(temp)
        if len(x_temp)>=vac_con_width/resolution[ndir-1]:
            print("|---> Sufficient vacuum region about %5.3f angstrom width was found."%(len(x_temp)*resolution[ndir-1]),file=log)
        else: 
            print("|---> Insufficient vacuum region about %5.3f angstrom. Temporarily, this narrow flat region was used as vacuum level. Please check about it."%(len(x_temp)*resolution[ndir-1]),file=log)
    elif n==1:
        for i in range(len(diff_x_temp[1:])):
            if diff_x_temp[i]!=1:
                discrete=i
        region_a=np.mean(x_temp[:discrete])
        region_b=np.mean(x_temp[discrete:])
        if values[0][3]=="Direct":
            x_model=np.mean(dir2car(coord,cell_vec))/resolution[ndir-1]
        else:
            x_model=np.mean(coord[:,2])/resolution[ndir-1]
        if abs(x_model-region_a) <= abs(x_model-region_b):
            region=np.array(x_temp[:discrete],dtype='d')
            E_region=np.array(temp[:discrete],dtype='d')
        else:
            region=np.array(x_temp[discrete:],dtype='d')
            E_region=np.array(temp[discrete:],dtype='d')
        E_vac=np.mean(E_region)
        print("|---> 2 Flat regions were found. Maybe dipole correction was performed. Please check about it.", file=log)
        print("|---> If you turn on visualization by -v tags, you can see blue lines. That is the selected region.", file=log)
    else:
        E_vac=max(temp)
        print("|---> Too many flat regions were found. Please check the convergence of Vacuum. For now, max energy is used.", file=log)


    #------------------------------------------------------------------
    # Shifting potential values with respect to E_fermi (default E_fermi = 0.0)
    #------------------------------------------------------------------
    shifting=True            
    if fermi_e+1==1:
        try:
            # fermi_e = float(extract('efermi','vasprun.xml')[2])
            fermi_e=float(extract('E-fermi','OUTCAR')[2])
            log.write("--------------------\n")
            log.write("Fermi energy= [ "+str(fermi_e)+" ] eV.\n")
            lLabel="Label left \"\Z24\F'Times New Roman'\\f02E\\f00\BPOT\M\Z24 (eV) (\\f02E\\f00-\\f02E\\f00\Bf\M)\"\n"
        except:
            shifting=False
            log.write("--------------------\n")
            log.write("OUTCAR file doesn't existed\n")
            log.write("[WARNING] Fermi energy is not extracted. Shifting is not done\n")
            print("OUTCAR file doesn't existed")
            print("[WARNING] fermi energy is not extracted. Shifting is not done")
            lLabel="Label left \"\Z24\F'Times New Roman'\\f02E\\f00\BPOT\M\Z24 (eV)\"\n"
    else:
        log.write("--------------------\n")
        print("OUTCAR file doesn't existed")
        print("(Manual setting) Fermi Energy is successfully set.")
        log.write("OUTCAR file doesn't existed\n")
        log.write("(Manually set) Fermi energy= ["+str(fermi_e)+" ] eV.\n")
        log.write("--------------------\n")
        lLabel="Label left \"\Z24\F'Times New Roman'\\f02E\\f00\BPOT\M\Z24 (eV) (\\f02E\\f00-\\f02E\\f00\Bf\M)\"\n"
    shifted_potavg=potavg-fermi_e

    #------------------------------------------------------------------
    # Calculate Workfunction Value. (Workfunction = Vacuum Level - Fermi Level)
    #------------------------------------------------------------------

    log.write("Vacuum Level is [ %5.10s ] eV.\n"%(E_vac))
    if shifting:
        log.write("So Workfunction is [ %5.10s ] eV.\n"%(E_vac-fermi_e))
    else:
        log.write("Workfunction = [ None ]\n")
    log.write("--------------------\n")



    #------------------------------------------------------------------
    # Visualization -- Total
    #------------------------------------------------------------------
    index=np.arange(1,len(potavg)+1,1)
    if direction == "Z":
        real_index=index*cell_vec_length[2]/float(len(potavg))
    elif direction == "Y" :
        real_index=index*cell_vec_length[1]/float(len(potavg))
    else:
        # direction == "X"
        real_index=index*cell_vec_length[0]/float(len(potavg))

    #------------------------------------------------------------------
    # Successfully Ended.
    #------------------------------------------------------------------    
    print("--------------         Workfunction Calculation is Done.        --------------")
    print("-------------- Please kindly look at [ workfunction.log ] file. --------------")
    outdat=open(directory+"/"+output_file,'w')
    for (x1,y1) in zip(index,shifted_potavg):
        print((str(x1)+" "+str(y1)),file=outdat)
    outdat.close()
       
    
    #------------------------------------------------------------------
    # Visualization (1) Igor
    #------------------------------------------------------------------
    if igor==True:
        if ".itx" not in igor_output:
            igor_output+=".itx"
        #lLabel is written in upper parts. Since it must be differnent case by case.
        bLabel="Label bottom \"Distance along \\f02%s\\f00 (\\{num2char(197)})\"\n"%(direction.lower())
        write_Igor2d(real_index,shifted_potavg,lLabel,bLabel,directory+"/"+igor_output,"Ep")
        log.write("Successfully finished writing itx file which name is [ %s ]"%(igor_output))
        
    #------------------------------------------------------------------
    # Visualization (2) matplotlib.pyplot
    #------------------------------------------------------------------
    import matplotlib.pyplot as plt
    X, Y = real_index, shifted_potavg
    plt.plot(X,Y,'r')
    if n!=0:
        # If there are dipole corrections
        selected_X, selected_Y = region, E_region-fermi_e
        plt.plot(selected_X*resolution[ndir-1],selected_Y,'bo')
    plt.savefig(directory+"/"+'Potential.eps')
    if not(visualization):
        import matplotlib
        matplotlib.use("Agg")
    else:
        plt.show()
    
    log.close()

#------------------------------------------------------------------
# Not ended well
#------------------------------------------------------------------
else:
    log.write(input_file+" doesn't existed... Just Ended")
    log.close()

--------------        LOCPOT exists. Calculation starts.        --------------
* Name of System : unknown system                          

* Matrix : [ 64 ] x [ 64 ] x [ 400 ]
--------------         Workfunction Calculation is Done.        --------------
-------------- Please kindly look at [ workfunction.log ] file. --------------


NameError: name 'region' is not defined

In [4]:
x_temp

array([218., 219., 220., 221., 222., 223., 224., 225., 227., 229., 231.])

In [5]:
temp

[5.805739968256323,
 5.806675011193481,
 5.806625554645312,
 5.806873843272411,
 5.806261398705493,
 5.806043107445239,
 5.8050501957150145,
 5.804515721968482,
 5.802518320449023,
 5.800206295355323,
 5.797686507862207]

In [7]:
diff_x=abs(x_temp-np.append(np.array([0],dtype='d'),x_temp[:-1]))

In [8]:
diff_x

array([218.,   1.,   1.,   1.,   1.,   1.,   1.,   1.,   2.,   2.,   2.])

In [6]:
diff_potavg

array([1.40841360e+01, 1.75262600e-01, 5.19065798e-01, 8.30760691e-01,
       1.10298072e+00, 1.31346138e+00, 1.46233388e+00, 1.53440981e+00,
       1.54415121e+00, 1.49262034e+00, 1.38869393e+00, 1.17128203e+00,
       9.30470066e-01, 5.88089807e-01, 3.38450625e-01, 1.41287394e-01,
       5.33934763e-02, 2.53882158e-01, 4.46828671e-01, 7.45695650e-01,
       1.05351287e+00, 1.27500180e+00, 1.45655696e+00, 1.52222547e+00,
       1.55368444e+00, 1.51210841e+00, 1.41281146e+00, 1.23527173e+00,
       1.00423959e+00, 7.14150278e-01, 3.92944598e-01, 4.43036997e-02,
       3.01667058e-01, 6.36522866e-01, 9.33683338e-01, 1.18655728e+00,
       1.37682705e+00, 1.50170846e+00, 1.55529824e+00, 1.54438412e+00,
       1.48553851e+00, 1.34256113e+00, 1.11648287e+00, 8.46479569e-01,
       5.09288459e-01, 3.05163068e-01, 9.93599103e-02, 8.81551932e-02,
       2.94110282e-01, 4.96568216e-01, 8.22553121e-01, 1.10065184e+00,
       1.32591376e+00, 1.47612614e+00, 1.53828206e+00, 1.55194345e+00,
      